In [55]:
!pip install scikit-learn gensim nltk
import os
import glob
import pickle
import nltk
import pandas as pd
import re
from sklearn.datasets import fetch_20newsgroups

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
import pickle
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import itertools
import matplotlib.pyplot as plt

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [56]:
newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
df = pd.DataFrame({'text': newsgroups.data, 'label': newsgroups.target})
df['label'] = df['label'].map(lambda x: newsgroups.target_names[x])
df

,text,label
0,\n\nI am sure some bashers of Pens fans are pr...,rec.sport.hockey
1,My brother is in the market for a high-perform...,comp.sys.ibm.pc.hardware
2,\n\n\n\n\tFinally you said what you dream abou...,talk.politics.mideast
3,\nThink!\n\nIt's the SCSI card doing the DMA t...,comp.sys.ibm.pc.hardware
4,1) I have an old Jasmine drive which I cann...,comp.sys.mac.hardware
...,...,...
18841,DN> From: nyeda@cnsvax.uwec.edu (David Nye)\nD...,sci.med
18842,\nNot in isolated ground recepticles (usually ...,sci.electronics
18843,I just installed a DX2-66 CPU in a clone mothe...,comp.sys.ibm.pc.hardware
18844,\nWouldn't this require a hyper-sphere. In 3-...,comp.graphics


In [57]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

def preprocess(text):
  tokens = word_tokenize(text.lower())
  text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
  text = re.sub(r'\S+@\S+', '', text)
  text = re.sub(r'[^\w\s]', '', text)
  text = re.sub(r'[^A-Za-z ]+', '', text)
  tokens = [word for word in tokens if word not in stop_words and len(word)>=3]
  tokens = [lemmatizer.lemmatize(word) for word in tokens]
  processed_text = ' '.join(tokens)
  return processed_text
df['processed_text'] = df['text'].apply(lambda x: preprocess(x))
df

,text,label,processed_text
0,\n\nI am sure some bashers of Pens fans are pr...,rec.sport.hockey,sure bashers pen fan pretty confused lack kind...
1,My brother is in the market for a high-perform...,comp.sys.ibm.pc.hardware,brother market high-performance video card sup...
2,\n\n\n\n\tFinally you said what you dream abou...,talk.politics.mideast,finally said dream mediterranean new .... area...
3,\nThink!\n\nIt's the SCSI card doing the DMA t...,comp.sys.ibm.pc.hardware,think scsi card dma transfer disk ... scsi car...
4,1) I have an old Jasmine drive which I cann...,comp.sys.mac.hardware,old jasmine drive use new system understanding...
...,...,...,...
18841,DN> From: nyeda@cnsvax.uwec.edu (David Nye)\nD...,sci.med,nyeda cnsvax.uwec.edu david nye neurology cons...
18842,\nNot in isolated ground recepticles (usually ...,sci.electronics,isolated ground recepticles usually unusual co...
18843,I just installed a DX2-66 CPU in a clone mothe...,comp.sys.ibm.pc.hardware,installed dx2-66 cpu clone motherboard tried m...
18844,\nWouldn't this require a hyper-sphere. In 3-...,comp.graphics,would n't require hyper-sphere 3-space point s...


In [58]:
df = df.drop('label',axis=1)
df

,text,processed_text
0,\n\nI am sure some bashers of Pens fans are pr...,sure bashers pen fan pretty confused lack kind...
1,My brother is in the market for a high-perform...,brother market high-performance video card sup...
2,\n\n\n\n\tFinally you said what you dream abou...,finally said dream mediterranean new .... area...
3,\nThink!\n\nIt's the SCSI card doing the DMA t...,think scsi card dma transfer disk ... scsi car...
4,1) I have an old Jasmine drive which I cann...,old jasmine drive use new system understanding...
...,...,...
18841,DN> From: nyeda@cnsvax.uwec.edu (David Nye)\nD...,nyeda cnsvax.uwec.edu david nye neurology cons...
18842,\nNot in isolated ground recepticles (usually ...,isolated ground recepticles usually unusual co...
18843,I just installed a DX2-66 CPU in a clone mothe...,installed dx2-66 cpu clone motherboard tried m...
18844,\nWouldn't this require a hyper-sphere. In 3-...,would n't require hyper-sphere 3-space point s...


In [60]:
df['processed']= df['processed_text'].apply(lambda x: [item for item in str(x).split()])
df = df.drop('processed_text', axis = 1)
df

,text,processed
0,\n\nI am sure some bashers of Pens fans are pr...,"[sure, bashers, pen, fan, pretty, confused, la..."
1,My brother is in the market for a high-perform...,"[brother, market, high-performance, video, car..."
2,\n\n\n\n\tFinally you said what you dream abou...,"[finally, said, dream, mediterranean, new, ......"
3,\nThink!\n\nIt's the SCSI card doing the DMA t...,"[think, scsi, card, dma, transfer, disk, ..., ..."
4,1) I have an old Jasmine drive which I cann...,"[old, jasmine, drive, use, new, system, unders..."
...,...,...
18841,DN> From: nyeda@cnsvax.uwec.edu (David Nye)\nD...,"[nyeda, cnsvax.uwec.edu, david, nye, neurology..."
18842,\nNot in isolated ground recepticles (usually ...,"[isolated, ground, recepticles, usually, unusu..."
18843,I just installed a DX2-66 CPU in a clone mothe...,"[installed, dx2-66, cpu, clone, motherboard, t..."
18844,\nWouldn't this require a hyper-sphere. In 3-...,"[would, n't, require, hyper-sphere, 3-space, p..."


In [61]:
df.to_csv('preprocessed_final.csv', index = False)

In [66]:
def evaluation_doc2vec(dataframe, params, processed_corpus, top=10):
    tagged_corpus = [TaggedDocument(doc, [i]) for i, doc in enumerate(processed_corpus)]

    score_list = []
    for param in params:
        param['score'] = 0
        try:
            model = Doc2Vec(tagged_corpus,
                            vector_size=param['vector_size'],
                            window=param['window'],
                            min_count=param['min_count'],
                            epochs=10)

            for i, row in dataframe.iterrows():
                new_doc = row['processed']
                test_doc_vector = model.infer_vector(new_doc)
                sims = model.dv.most_similar(positive=[test_doc_vector], topn=top)

                for i, (sim_index, sim_score) in enumerate(sims):
                    if row['processed'] == dataframe['processed'].iloc[sim_index]:
                        param['score'] += (top - i)
            score_list.append(param)
        except Exception as error:
            print(f'Cannot evaluate model with parameters {param} because of error: {error}')
            continue
    return score_list

vector_size_values = [100, 150, 200, 250, 300]
window_values = [5, 10]
min_count_values = [1, 3, 5]

# Create all possible combinations of hyperparameters using itertools.product
param_combinations = list(itertools.product(vector_size_values, window_values, min_count_values))
params = [{'vector_size': vector_size, 'window': window, 'min_count': min_count} for vector_size, window, min_count in param_combinations]

score_list = evaluation_doc2vec(df, params, df['processed'])

best_params = max(score_list, key=lambda x: x['score'])
print("Best Parameters:", best_params)
print("Best Score:", best_params['score'])

for param in score_list:
    print(f"Parameters: {param} Score: {param['score']}")

Best Parameters: {'vector_size': 200, 'window': 5, 'min_count': 3, 'score': 188750}
Best Score: 188750
Parameters: {'vector_size': 100, 'window': 5, 'min_count': 1, 'score': 185104} Score: 185104
Parameters: {'vector_size': 100, 'window': 5, 'min_count': 3, 'score': 187206} Score: 187206
Parameters: {'vector_size': 100, 'window': 5, 'min_count': 5, 'score': 187104} Score: 187104
Parameters: {'vector_size': 100, 'window': 10, 'min_count': 1, 'score': 177302} Score: 177302
Parameters: {'vector_size': 100, 'window': 10, 'min_count': 3, 'score': 180393} Score: 180393
Parameters: {'vector_size': 100, 'window': 10, 'min_count': 5, 'score': 181158} Score: 181158
Parameters: {'vector_size': 150, 'window': 5, 'min_count': 1, 'score': 184984} Score: 184984
Parameters: {'vector_size': 150, 'window': 5, 'min_count': 3, 'score': 187610} Score: 187610
Parameters: {'vector_size': 150, 'window': 5, 'min_count': 5, 'score': 188109} Score: 188109
Parameters: {'vector_size': 150, 'window': 10, 'min_count

In [69]:
%%time
tagged_corpus = [TaggedDocument(doc, [i]) for i, doc in enumerate(df['processed'])]

model = Doc2Vec(tagged_corpus, vector_size=200, window=5, min_count=3, epochs=10)
model.save("doc2vec_final.model")

model = Doc2Vec.load("doc2vec_final.model")

CPU times: user 1min 55s, sys: 2.38 s, total: 1min 57s
Wall time: 1min 15s


In [70]:
def vectorize_doc2vec(corpus, model):
    doc_vectors = []
    for doc in corpus:
        vector = model.infer_vector(doc)
        doc_vectors.append(vector)
    return doc_vectors

vectorized_docs = vectorize_doc2vec(df['processed'], model)

doc_vectors_df = pd.DataFrame(vectorized_docs)
doc_vectors_df.to_csv('doc2vec_vectors_final.csv')

In [71]:
doc_vectors_df

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.096245,0.003599,-0.085176,0.230895,0.064714,0.058563,0.162013,0.053399,-0.078224,0.127144,...,-0.068914,0.165855,-0.139714,0.018813,0.186408,0.246452,0.127322,0.037057,0.035583,-0.192175
1,-0.079716,-0.090094,0.003842,-0.029230,0.091268,0.080343,-0.024272,0.188793,-0.036782,0.024933,...,0.164396,-0.031148,-0.016781,-0.069668,0.120217,0.086646,0.081417,-0.013408,-0.056917,-0.052905
2,0.239040,-0.084714,0.090230,0.242841,0.270138,0.399826,0.006096,0.441832,-0.312696,-0.138106,...,-0.125470,0.051412,0.059226,-0.057874,0.426521,-0.072455,0.080208,0.068673,-0.240681,-0.269405
3,0.299779,-0.013954,0.210941,-0.128154,0.116408,0.073526,-0.098613,0.179598,0.062506,0.072558,...,0.310030,-0.218989,-0.077160,-0.038774,0.162210,-0.052915,0.167200,0.059254,0.039290,0.134820
4,-0.064090,0.016694,0.125658,-0.389051,0.163018,0.150782,0.003218,-0.053042,-0.008400,0.007497,...,0.268552,0.212245,0.184841,0.022134,0.108727,-0.105548,0.056528,0.068675,0.005168,0.110122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18841,0.070677,0.038380,-0.235518,0.111217,0.207379,-0.128073,-0.454795,0.214764,0.167244,0.050483,...,0.222806,-0.109211,-0.189658,-0.095949,0.158829,-0.043469,-0.246037,-0.251869,-0.313604,0.176173
18842,0.033007,0.062806,-0.012405,-0.061101,0.078836,0.019527,0.063720,0.056288,-0.095236,0.008396,...,0.090953,-0.049006,-0.017478,-0.056296,0.094059,0.049073,-0.005614,0.016835,0.053064,-0.011447
18843,0.037536,0.080192,-0.112573,0.026911,0.196090,-0.104824,-0.035324,0.111015,-0.219219,0.134587,...,0.053099,-0.034258,-0.093803,0.084292,0.202540,0.136905,0.073164,-0.150450,-0.080591,0.208803
18844,0.057380,-0.181662,0.074540,-0.037389,0.152723,0.080829,-0.009984,0.107742,0.075731,0.115721,...,0.066215,-0.060865,-0.037562,-0.087966,0.022240,0.081377,0.128182,-0.044847,-0.078601,-0.084428
